# Row Level Security

In addition to database access permissions available through ROLE and GRANT privilege system, YugabyteDB provides a more granular level security where tables can have row security policies that restrict rows users can access.

Row-level security (RLS) restricts rows that can be returned by normal queries or inserted, updated, or deleted by DML commands. RLS policies can be created specific to a DML command or with ALL commands. They can also be used to create policies on a particular role or multiple roles.

To start, connect to your YugabyteDB instance


## Connect to YugabyteDB using the PostgreSQL Driver for Python
The following cells requires:
- Python 3.8+ and psycopg2

In [ ]:
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_DATA_DDL_FILE
%store -r MY_DATA_DML_FILE

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

# engine = create_engine(connection_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_str}

To demonstrate row level security, you will create a test table with some data. The test table will be a table of employee data.

In [ ]:
%sql create table employees ( empno int, ename text, address text, salary int,account_number text );

%sql insert into employees values (1, 'joe', '56 grove st',  20000, 'AC-22001' );
%sql insert into employees values (2, 'mike', '129 81 st',  80000, 'AC-48901' );
%sql insert into employees values (3, 'julia', '1 finite loop',  40000, 'AC-77051');

Next, create three users, one for each of the employees added to the employees table.

In [ ]:
%sql create role joe login password 'yourpasswordhere';
%sql grant select on employees to joe;

%sql create role mike login password 'yourpasswordhere';
%sql grant select on employees to mike;

%sql create role julia login password 'yourpasswordhere';
%sql grant select on employees to julia;

You are going to create a row level security policy that limits what each of the roles can see in the table. To achieve this, you will use a `create policy` query.

In [ ]:
%sql create policy emp_rls_policy on employees for all to public using (ename=current_user);

The name of the created policy is emp_rls_policy. This policy is being applied to the employees table for all of the DDL commands. The public keyword indicates that this rule applies to all roles in the database. The policy will automatically filter the data so that the data shown is only the rows where the ename field matches the name of the current role.

To apply this to the table, you can alter the table to enabled row level security.



In [ ]:
alter table employees enable row level security;

To verify that this works correctly, connect to the database as one of the created roles and attempt to query the employees table.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # \d tbl_countriees

YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -U joe -d db_ybu -c "select * from employees;"